Задача с качеством уроков

В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы старательно записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки мы записываем в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

В этой задаче тебе нужно будет в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.

В архиве (https://yadi.sk/d/ESH_lU5sZRdKhA) содержится четыре файла с выгрузкой строк из базы.

lessons.txt содержит следующие поля:
    id – уникальный идентификатор урока.
    event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id. Подробнее про это будет позже
    subject – предмет урока, просто строка
    scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC.

quality.txt содержит следующие поля:
    lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)
    tech_quality – собственно оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

users.txt хранит информацию о пользователях и содержит всего два поля:
    id – уникальный идентификатор пользователя
    role – указывает является ли пользователь учеником (pupil) или учителем (tutor)

Файл participants.txt позволяет связать урок с его участниками. Он содержит следующие поля:
    user_id – идентификатор пользователя (указывает на запись в файле users.txt)
    event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).


In [1]:
import pandas as pd

In [2]:
with open('lessons.txt', encoding='utf8') as f: 
    lessons = pd.read_csv('lessons.txt',sep='|', header=None, names=["id", "event_id", "subject", "scheduled_time"]) 

In [3]:
lessons = lessons.drop([0, 1, 380])

In [4]:
lessons

,id,event_id,subject,scheduled_time
2,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
3,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00
4,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00
5,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
6,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00
...,...,...,...,...
375,f579351b-177c-4c1c-850a-58bbda8b0b22,48887,hist,2020-01-15 13:45:00
376,056c3d91-dce7-4071-91d3-1de816ee63a3,51001,hist,2020-01-17 12:45:00
377,c64a1daf-d7f5-4c8c-ac9f-df634d77327b,50304,bio,2020-01-11 11:00:00
378,aa7702bc-5271-469c-a1c2-b7e814fd7e20,49880,bio,2020-01-16 15:00:00


In [5]:
lessons = lessons.astype({'event_id': 'int64'});

In [6]:
lessons.dtypes

id                object
event_id           int64
subject           object
scheduled_time    object
dtype: object

In [7]:
with open('quality.txt', encoding='utf8') as f: 
    quality = pd.read_csv('quality.txt',sep='|', header=None, names=["lesson_id", "tech_quality"]) 

In [8]:
quality = quality.drop([0,1,367])

In [9]:
quality

,lesson_id,tech_quality
2,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5
3,62e1a078-33de-47c1-99d2-845b1daca56f,5
4,62e1a078-33de-47c1-99d2-845b1daca56f,5
5,00fc6685-f53a-49bb-b960-5e0042fd3852,5
6,00fc6685-f53a-49bb-b960-5e0042fd3852,5
...,...,...
362,c64a1daf-d7f5-4c8c-ac9f-df634d77327b,5
363,aa7702bc-5271-469c-a1c2-b7e814fd7e20,5
364,8643fa00-217a-42ae-a27d-e6f47c0501f1,5
365,7a15af58-39d0-4207-bfdf-3fa176b7fb4e,5


In [10]:
quality['tech_quality'] = pd.to_numeric(quality['tech_quality'], errors='coerce', downcast='unsigned')

In [11]:
with open('users.txt', encoding='utf8') as f: 
    users = pd.read_csv('users.txt',sep='|', header=None, names=["id", "role"]) 
users = users.drop([0,1,745])

In [12]:
users

,id,role
2,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
3,4df2832a-1d63-4453-9659-43993fc35996,tutor
4,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
5,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
6,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor
...,...,...
740,582e6abb-dd9a-42fa-b441-455eac28327e,pupil
741,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor
742,d3364cf3-46cc-4e64-ad11-70a069f7c049,pupil
743,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor


In [13]:
with open('participants.txt', encoding='utf8') as f: 
    participants = pd.read_csv('participants.txt',sep='|', header=None, names=["event_id", "user_id"]) 
participants = participants.drop([0,1,745])

In [14]:
participants

,event_id,user_id
2,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
3,38114,4df2832a-1d63-4453-9659-43993fc35996
4,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
5,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
6,52790,a1323b68-c82c-429a-8e2c-65597e648c1a
...,...,...
740,50304,ab7149fc-b650-4b6b-a368-0581f78276ea
741,49880,8476b612-223f-4c86-9b0b-14bc04759233
742,49880,582e6abb-dd9a-42fa-b441-455eac28327e
743,55449,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb


In [15]:
participants = participants.astype({'event_id': 'int64'});

In [16]:
users['role'].unique()

array([' pupil', ' tutor', ' admin'], dtype=object)

In [36]:
lessons_phys = lessons.loc[lessons['subject'] == ' phys    ']
lessons_phys

,id,event_id,subject,scheduled_time
2,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
5,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
6,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00
8,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 15:00:00
10,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 14:00:00
...,...,...,...,...
368,e90f529b-1061-4efd-9bba-f73ecee2ef27,55490,phys,2020-01-19 08:00:00
369,f0002160-e57e-4e90-ac7e-c8fcdbbae26f,41963,phys,2020-01-19 09:00:00
370,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,55881,phys,2020-01-19 15:00:00
371,457accf4-bd99-4b9d-b540-ef200e65afbd,49007,phys,2020-01-19 05:00:00


2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.


3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67 2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25
...
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.